
<img align="left" src="https://earthdata.nasa.gov/img/earthdata-fb-image.jpg" width="250">

# __Amazon river freshwater discharge impacts on coastal waters:__

## __Hands-on tutorial of AWS in-region access of NASA Earthdata products__


This notebook provides a basic end-to-end workflow to interact with data "in-place" from the NASA Earthdata Cloud, by accessing AWS S3 locations provided by [NASA Harmony](http://harmony.earthdata.nasa.gov/) outputs without the need to download data. While these outputs can be downloaded locally, the cloud offers the ability to scale compute resources to perform analyses over large areas and time spans, which is critical as data volumes continue to grow. 

This workflow combines search, discovery, access, reformatting, basic analyses, and plotting components presented during Part-II. Though the example we're working with in this notebook only focuses on a small time and area to account for a large number of concurrent processing requests, this workflow can be modified and scaled up to suit a larger time range and region of interest. 

#### Learning objectives:

- Understand the Pangeo BinderHub environment used during the workshop and how to execute code within a Jupyter Notebook
- Search for Liquid Water Equivalent (LWE) data from GRACE/GRACE-FO and Sea Surface Salinity (SSS) from SMAP 
- Execute programmatic data access queries, plotting, and direct in-region cloud access using open source Python libraries.
- Access data in Zarr format from Earthdata Cloud (AWS)
- Subset both, plot and compare coincident data.
- Identify resources, including the Earthdata Cloud Primer, for getting started with Amazon Web Services outside of the Workshop to access and work with data with a cloud environment.


___

<p float="left">
    <img src="https://jupyter.org/assets/main-logo.svg" width="100">
    <img src="https://github.com/pangeo-data/pangeo/raw/master/docs/_static/pangeo_simple_logo.svg" width="200">
    
</p>




### __Pangeo BinderHub and Project Jupyter__

First, some basics on the Pangeo compute environment used during the live workshop and how to interact with Jupyter Notebooks and the Jupyter Lab interface.

* [Pangeo BinderHub](https://binder.pangeo.io/): A multi-user server for interactive data analysis. This Hub is running in the AWS `us-west-2` region, which is where all Earthdata Cloud data and transformation service outputs are located. Pangeo is supported, in part, by the National Science Foundation (NSF) and the National Aeronautics and Space Administration (NASA). Google provided compute credits on Google Compute Engine. The Pangeo community promotes open, reproducible, and scalable science. We thank you for supporting this AGU Workshop.

**This Hub is only supported during the live AGU workshop**. See instructions at the bottom of this notebook for how to set up your own AWS EC2 instance so that you can perform the same cloud access within your personal AWS environment. 

* [Jupyter Notebook](https://jupyter-notebook.readthedocs.io/en/latest/): Interactive, reproducible, open source, and exploratory browser integrated computing environment.
* [JupyterLab](https://github.com/jupyterlab/jupyterlab): Web-based integrated IDE for computational workflows.

### __Jupyter Notebook Basics__

The body of a notebook is composed of cells. Each cell contains either markdown, code input, code output, or raw text. Cells can be included in any order and edited and executed at-will.

**Markdown cells** - These are used to build a nicely formatted narrative around the code in the document.

**Code cells** - These are used to define the computational code in the document. They come in two forms: the input cell where the user types the code to be executed, and the output cell which is the representation of the executed code.

**Raw cells** - These are used when text needs to be included in raw form, without execution or transformation.

#### Execute a cell or selected cells by pressing shift + enter


In [ ]:
print('Hello World!')

#### Collapse a cell or cell output by clicking on the blue line to the left of the cell

The cell content is replaced by three dots, indicating that the cell is collapsed.

#### Execute multiple cells or run the entire notebook
Select cells with **shift + Up** or **shift + Down** and then execute selection with **shift + enter**.

#### Run the whole notebook in a single step by clicking on the menu Run -> Run All Cells.

See https://jupyter.readthedocs.io/en/latest/running.html for more guidance on running notebooks.

___
### __Import modules__

The Python ecosystem is organized into modules.  A module must be imported before the contents of that modules can be used.  It is good practice to import modules in the first code cell of a notebook or at the top of your script.  Not only does this make it clear which modules are being used, but it also ensures that the code fails at the beginning because one of the modules is not installed rather half way through after crunching a load of data.

For some modules, it is common practice to shorten the module names according to accepted conventions.  For example, the plotting module `matplotlib.pyplot` is shortened to `plt`.  It is best to stick to these conventions rather than making up your own short names so that people reading your code see immediately what you are doing. 

In [ ]:
from netrc import netrc
from platform import system
from getpass import getpass
from urllib import request
from http.cookiejar import CookieJar
from os.path import join, expanduser
from pprint import pprint
import intake
import s3fs
import rasterio
import zarr
import matplotlib.pyplot as plt
import dask.array as da
import time
import requests
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.animation as animation
import cartopy.crs as ccrs
import cartopy
import zarr
import s3fs
from IPython.display import HTML
import json


%matplotlib inline

## __Earthdata Login__

An Earthdata Login account is required to access data, as well as discover restricted data, from the NASA Earthdata system. Please visit https://urs.earthdata.nasa.gov to register and manage your Earthdata Login account. This account is free to create and only takes a moment to set up.

At this point in time (as we are still transitioning to a cloud environment), in order to access data from the Earthdata Cloud, you need special, early access, persmissions. For the workshop today, you have already been added to the list and the Earthdata login you provided prior to the workshop has been granted this access.

The `setup_earthdata_login_auth` function will allow Python scripts to log into any Earthdata Login application programmatically.  To avoid being prompted for
credentials every time you run and also allow clients such as curl to log in, you can add the following
to a `.netrc` (`_netrc` on Windows) file in your home directory:

```
machine urs.earthdata.nasa.gov
    login <your username>
    password <your password>
```

Make sure that this file is only readable by the current user or you will receive an error stating
"netrc access too permissive."

`$ chmod 0600 ~/.netrc` 

In [ ]:
TOKEN_DATA = ("<token>"
              "<username>%s</username>"
              "<password>%s</password>"
              "<client_id>PODAAC CMR Client</client_id>"
              "<user_ip_address>%s</user_ip_address>"
              "</token>")


def setup_cmr_token_auth(endpoint: str='cmr.earthdata.nasa.gov'):
    ip = requests.get("https://ipinfo.io/ip").text.strip()
    return requests.post(
        url="https://%s/legacy-services/rest/tokens" % endpoint,
        data=TOKEN_DATA % (input("Username: "), getpass("Password: "), ip),
        headers={'Content-Type': 'application/xml', 'Accept': 'application/json'}
    ).json()['token']['id']


def setup_earthdata_login_auth(endpoint: str='urs.earthdata.nasa.gov'):
    netrc_name = "_netrc" if system()=="Windows" else ".netrc"
    try:
        username, _, password = netrc(file=join(expanduser('~'), netrc_name)).authenticators(endpoint)
    except (FileNotFoundError, TypeError):
        print('Please provide your Earthdata Login credentials for access.')
        print('Your info will only be passed to %s and will not be exposed in Jupyter.' % (endpoint))
        username = input('Username: ')
        password = getpass('Password: ')
    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, endpoint, username, password)
    auth = request.HTTPBasicAuthHandler(manager)
    jar = CookieJar()
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener)


# Get your authentication token for searching restricted records in the CMR:
_token = setup_cmr_token_auth(endpoint="cmr.earthdata.nasa.gov")

# Start authenticated session with URS to allow restricted data downloads:
setup_earthdata_login_auth(endpoint="urs.earthdata.nasa.gov")

___
## __Data Search and Discovery__

Background on the two datasets we're working with in this tutorial:

[**JPL GRACE and GRACE-FO Mascon Ocean, Ice, and Hydrology Equivalent Water Height Coastal Resolution Improvement (CRI) Filtered Release 06 Version 02**](https://podaac.jpl.nasa.gov/dataset/TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06_V2)

Provides land water equivalent (LWE) thickness for observing seasonal changes in water storage around the river. When discharge is high, the change in water storage will increase, pointing to a wet season. This product provides gridded monthly global water storage/height anomalies in a single data file in netCDF format, and can be used for analysis for ocean, ice, and hydrology phenomena. Source data are from [GRACE](https://podaac.jpl.nasa.gov/GRACE) and [GRACE-FO](https://podaac.jpl.nasa.gov/GRACE-FO)

[**RSS SMAP Level 3 Sea Surface Salinity Standard Mapped Image 8-Day Running Mean V4.0 Validated Dataset**](https://podaac.jpl.nasa.gov/SMAP?sections=data)

Daily data files (NetCDF format) for this product are based on Sea Surface Salinity averages spanning an 8-day moving time window, gridded at 0.25 degree x 0.25 degree.

#### First, define the region of interest over the Amazon river basin and set the temporal range for the year 2019:

In [ ]:
# Bounding Box spatial parameter in decimal degree 'W,S,E,N' format.
bounding_box = '-52,-2,-43,6'

# Each date in yyyy-MM-ddTHH:mm:ssZ format; date range in start,end format
temporal = '2019-01-01T00:00:00Z,2019-12-31T23:59:59Z'

#### Set up a nested dictionary with the two data products of interest

Before we search programmatically using the [Common Metadata Repository (CMR)](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html), we can use NASA Earthdata Search to visualize file coverage over multiple data sets and to access the same data you will be working with below: 
[Earthdata Search project](https://search.earthdata.nasa.gov/projects?p=!C1938032626-POCLOUD!C1940468263-POCLOUD!C1650311642-PODAAC!C1664148252-PODAAC&pg[1][v]=t&pg[1][gsk]=-start_date&pg[2][v]=t&pg[2][gsk]=-start_date&pg[3][v]=t&pg[4][v]=t&q=TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06_V2&sb[0]=-52%2C-2%2C-43%2C6&m=2.00390625!-47.50048828125!6!1!0!0%2C2&qt=2019-04-01T00%3A00%3A00.000Z%2C2019-04-30T23%3A59%3A59.999Z&tl=1591244551!4!!)

In [ ]:
search_parameters = { 
    'grace': {
        'short_name': 'TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06_V2',
        'provider': 'POCLOUD',
        'bounding_box': bounding_box,
        'temporal': temporal,
        'token': _token},
    'smap': {
        'short_name': 'SMAP_RSS_L3_SSS_SMI_8DAY-RUNNINGMEAN_V4',
        'provider': 'POCLOUD',
        'bounding_box': bounding_box,
        'temporal': temporal,
        'token': _token},
            }

#### Discover file number and file size 

Using CMR search, determine the number of files that exist over this time and area of interest, as well as the average size and total volume of those files.

In [ ]:
search_url = "https://cmr.earthdata.nasa.gov/search/granules"
output_format="json"

for k, v in search_parameters.items(): #fn.search_granules(search_parameters[k], _token)
    parameters = {
        "scroll": "true",
        "page_size": 100,
    }

    response = requests.post(f"{search_url}.{output_format}", params=parameters, data=v)
    response.raise_for_status()

    hits = int(response.headers['CMR-Hits'])
    if hits > 0:
        print(f"Found {hits} files")
        results = json.loads(response.content)
        granules = []
        granules.extend(results['feed']['entry'])
        granule_sizes = [float(granule['granule_size']) for granule in granules]
        print(f"The total size of all files is {sum(granule_sizes):.2f} MB")
    else:
        print("Found no hits")

#### Locate the data access URLs provided by the data product (collection) metadata:

GRACE data:

In [ ]:
r = requests.get(url="https://cmr.earthdata.nasa.gov/search/granules.umm_json", 
                 params=search_parameters['grace'])
grace_gran = r.json()
print("files returned:",grace_gran['hits'])
grace_gran['items'][0]['umm']['RelatedUrls']

and SMAP data:

In [ ]:
r = requests.get(url="https://cmr.earthdata.nasa.gov/search/granules.umm_json", 
                 params=search_parameters['smap'])

smap_gran = r.json()
print("files returned:",smap_gran['hits'])
smap_gran['items'][0]['umm']['RelatedUrls']

___
## __Data Access from NASA Earthdata Harmony__

As you have seen so far in this workshop, there are several different methodologies to search and access data. The URLs above can be located with the data granule (file) metadata, and pulled into a list that can then be bulk downloaded as you've seen in the previous tutorials.

[Harmony](https://harmony.earthdata.nasa.gov/) is a growing effort across NASA EOSDIS community to provide a consistent method to access and perform data subsetting and transformation services for data in the Earthdata Cloud. These services are processed in the cloud, with data archived in the cloud, and outputs can be accessed by downloading to a local machine or through direct in-region access via Amazon Web Services. These next steps walk through this access method to access the data outputs directly within the AWS `us-west-2` region so that we can read the data into memory within this AWS hub environment. 

#### __Get collection ID for Harmony__

Harmony operates on an input Collection concept-id (a CMR construct). Here's how you identify the CMR concept-id for the data products of interest:

In [ ]:
r = requests.get(url="https://cmr.earthdata.nasa.gov/search/collections.umm_json", 
                 params=search_parameters['grace'])

grace_coll = r.json()
grace_coll['hits']
grace_coll_meta = grace_coll['items'][0]['meta']
grace_coll_id = grace_coll_meta['concept-id']
print('GRACE collection id:', grace_coll_id)

r = requests.get(url="https://cmr.earthdata.nasa.gov/search/collections.umm_json", 
                 params=search_parameters['smap'])

smap_coll = r.json()
smap_coll['hits']

smap_coll_meta = smap_coll['items'][0]['meta']
smap_coll_id = smap_coll_meta['concept-id']
print('SMAP collection id:', smap_coll_id)

#### __Request SMAP data reformatted to Zarr__:

[Zarr](https://zarr.readthedocs.io/en/stable/) is a format for the storage of chunked, compressed, N-dimensional arrays. Zarr also enables you to store arrays in memory, on disk, inside a Zip file, or on S3. Harmony's Zarr Reformatter service will transform the SMAP data from its native NetCDF format to Zarr, to allow us to open and download/read just the data that we require for our Amazon Basin study area.


For our science use case, ideally we'd also want to request an entire year but since this can take a long time to process with many concurrent requests during this live workshop, we will just request a month of data for demonstration purposes.

In [ ]:
harmony_root = 'https://harmony.earthdata.nasa.gov'
harmony_params = {
    'collection_id': smap_coll_id,
    'ogc-api-coverages_version': '1.0.0',
    'variable': 'all',
    'lat':'(-2:6)',
    'lon':'(-52:-43)',
    'start': '2019-04-01T00:00:00Z',
    'stop':'2019-04-30T23:59:59Z',
    'format': 'application/x-zarr',
}

smap_url = harmony_root+'/{collection_id}/ogc-api-coverages/{ogc-api-coverages_version}/collections/{variable}/coverage/rangeset?format={format}&subset=lat{lat}&subset=lon{lon}&subset=time("{start}":"{stop}")'.format(**harmony_params)
print(smap_url)

In [ ]:
smap_response = request.urlopen(smap_url)
smap_results = smap_response.read()
smap_json = json.loads(smap_results)
print(json.dumps(smap_json, indent=2))
smap_jobId = smap_json['jobID']

In [ ]:
smap_job_url = f'https://harmony.earthdata.nasa.gov/jobs/{smap_jobId}'

while True:
    loop_response = request.urlopen(smap_job_url)
    loop_results = loop_response.read()
    job_json = json.loads(loop_results)
    if job_json['status'] != 'running':
        break
    print(f"# Job status is running. Progress is {job_json['progress']} %. Trying again.")
    time.sleep(10)

smap_links = []
if job_json['status'] == 'successful' and job_json['progress'] == 100:
    print("# Job progress is 100%. Links to job outputs are displayed below:")
    smap_links = [link['href'] for link in job_json['links']]
    display(smap_links)

In [ ]:
smap_zarr_urls = smap_links[5:]
smap_zarr_urls

**Access credentials for the output zarr file**

Credentials provided in the Harmony job response provide authenticated access to your staged S3 resources.

Grab the credentials as a JSON string, load to a Python dictionary, and display their expiration date:

In [ ]:
with request.urlopen(f"https://harmony.earthdata.nasa.gov/cloud-access") as f:
    creds = json.loads(f.read())

creds['Expiration']

### Open staged zarr files with *s3fs*

We use the AWS `s3fs` package to get metadata about the zarr data store and read in the credentials we pulled from our Harmony job response:

In [ ]:
zarr_fs = s3fs.S3FileSystem(
    key=creds['AccessKeyId'],
    secret=creds['SecretAccessKey'],
    token=creds['SessionToken'],
    client_kwargs={'region_name':'us-west-2'},
)

#### SMAP loaded into xarray

 `xarray` is a python package designed to work with multi-dimensional arrays.  See the [xarray website](http://xarray.pydata.org/en/stable/) for more information. This next code block will take all of the month's worth of SMAP data we pulled above from Harmony into xarray in a single command.

In [ ]:
smap_zarr_urls = smap_links[5:]
smap_zarr_stores = [zarr_fs.get_mapper(root=u, check=False) for u in smap_zarr_urls]
ds_SMAP = xr.open_mfdataset(smap_zarr_stores, engine="zarr")

print(ds_SMAP)

____
### __We'll come back to our SMAP data now that it's loaded into xarray.__

**We'll now move into breakout rooms to work through the same data acces steps as above with the GRACE data.**

As you run through each of the code blocks, worth within your groups to ask questions and discuss the workflow. Instructors will move in and out of breakout rooms to offer assistance. 

#### __Request GRACE data reformatted to Zarr__:

In [ ]:
harmony_root = 'https://harmony.earthdata.nasa.gov'
harmony_params = {
    'collection_id': grace_coll_id,
    'ogc-api-coverages_version': '1.0.0',
    'variable': 'all',
    'lat':'(-2:6)',
    'lon':'(-52:-43)',
    'start': '2019-01-01T00:00:00Z',
    'stop':'2019-12-31T23:59:59Z',
    'format': 'application/x-zarr',
}

grace_url = harmony_root+'/{collection_id}/ogc-api-coverages/{ogc-api-coverages_version}/collections/{variable}/coverage/rangeset?format={format}&subset=lat{lat}&subset=lon{lon}&subset=time("{start}":"{stop}")'.format(**harmony_params)
print(grace_url)

In [ ]:
grace_response = request.urlopen(grace_url)
grace_results = grace_response.read()
grace_json = json.loads(grace_results)
print(json.dumps(grace_json, indent=2))
grace_jobId = grace_json['jobID']

In [ ]:
grace_job_url = f'https://harmony.earthdata.nasa.gov/jobs/{grace_jobId}'

while True:
    loop_response = request.urlopen(grace_job_url)
    loop_results = loop_response.read()
    job_json = json.loads(loop_results)
    if job_json['status'] != 'running':
        break
    print(f"# Job status is running. Progress is {job_json['progress']} %. Trying again.")
    time.sleep(10)

grace_links = []
if job_json['status'] == 'successful' and job_json['progress'] == 100:
    print("# Job progress is 100%. Links to job outputs are displayed below:")
    grace_links = [link['href'] for link in job_json['links']]
    display(grace_links)

#### __Access urls for the output zarr files__

The new zarr dataset is staged for us in an S3 bucket. The url is the last one in the list shown above.

Select the url and display below:

In [ ]:
grace_zarr_urls = grace_links[-1]
grace_zarr_urls

In [ ]:
grace_zarr_store = zarr_fs.get_mapper(root=grace_zarr_urls, check=False)
grace_zarr_dataset = zarr.open(grace_zarr_store)

print(grace_zarr_dataset.tree())

In [ ]:
print(grace_zarr_dataset.lwe_thickness.info)

#### __Open staged zarr file with *xarray*__

Read more about `xarray`'s zarr reader here: http://xarray.pydata.org/en/stable/generated/xarray.open_zarr.html

This xarray method allows you to pull in the Zarr outputs from Harmony directly into an xarray dataset. 

Open the zarr dataset and print the dataset "header":

In [ ]:
ds_GRACE = xr.open_zarr(grace_zarr_store)
print(ds_GRACE)

**Subset by Latitude/Longitude**

Once we have obtained all the data, to make processing quicker, we are going to subset datasets by latitude/longitude for the Amazon River estuary.

Once we have obtained the GRACE-FO data, we should spatial subset the data to the minimal area covering the Amazon River estuary. This will reduce processing load and reduce cloud costs for the user.

Make a GRACE-FO subset and display the min, max of the *lat* and *lon* variables:

In [ ]:
subset_GRACE = ds_GRACE.sel(lat=slice(-18, 10), lon=slice(275, 330))
print(subset_GRACE.lat.min().data, 
      subset_GRACE.lat.max().data,
      subset_GRACE.lon.min().data,
      subset_GRACE.lon.max().data)

**Select the variable for Land Water Equivalent Thickness (*lwe_thickness*)**

Grab the land water equivalent thickness variable from the GRACE subset:

In [ ]:
lwe = subset_GRACE.lwe_thickness
print(lwe)

### __Return to the main room__

Now that the GRACE data have also been loaded into xarray and subsetted, we'll return to the main room to begin our plotting and analysis.
____

## __Plotting and analysis__

__First we'll plot and animate the GRACE data over time, and then we'll subset and plot the SMAP data.__

Now that we have a time/space slice of interest from the GRACE collection, let's plot the first time step, to see what we've got (or to see what it looks like. First we'll define two functions to make the plotting (and next animation step) a bit more convenient:

In [ ]:
def setup_map(ax, pmap, ds_subset, x, y, var, t, cmap, levels, extent):
    title = str(pd.to_datetime(ds_subset.time[t].values))
    pmap.set_title(title, fontsize=14)
    pmap.coastlines()
    pmap.set_extent(extent)
    pmap.add_feature(cartopy.feature.RIVERS)
    variable_desired = var[t,:,:]
    cont = pmap.contourf(x, y, variable_desired, cmap=cmap, levels=levels, zorder=1)
    return cont

def animate_ts(framenumber, ax, pmap, ds_subset, x, y, var, t, cmap, levels, extent):
    cont = setup_map(ax, pmap, ds_subset, x, y, var, t + framenumber, cmap, levels, extent) 
    return cont

In [ ]:
# Initialize a matplotlib plot object and add subplot:
fig = plt.figure(figsize=[13,9]) 
ax = fig.add_subplot(1, 1, 1)

# Configure axes to display projected data using PlateCarree crs:
pmap = plt.axes(projection=ccrs.PlateCarree())

# Get arrays of x and y to label the plot axes:
x,y = np.meshgrid(subset_GRACE.lon, subset_GRACE.lat)                        

# Set a few constants for plotting the GRACE-FO time series:
time_start  = 168
cmap_name   = "bwr_r"
cmap_levels = np.linspace(-100., 100., 14)
map_extent  = [-85, -30, -16, 11]

# Plot the first timestep: 
cont = setup_map(ax, pmap, subset_GRACE, x, y, lwe, time_start, cmap_name, cmap_levels, map_extent)

fig.colorbar(cont, ticks=cmap_levels, orientation='horizontal', label='Land Water Equivalent Thickness (cm)')

#### Animate changes over time

Let's now explore how land water mass changes throughout the year 2019, by creating an animation of GRACE monthly land water equivalent (LWE) maps over the Amazon River.

Plot all the 2019 timesteps sequentially to create an animation of land water equivalent thickness for the Amazon Rainforest territories:

In [ ]:
ani = animation.FuncAnimation(fig, animate_ts, frames=range(0,12), fargs=(
    ax, pmap, subset_GRACE, x, y, lwe, time_start, cmap_name, cmap_levels, map_extent
), interval=500)

HTML(ani.to_html5_video())

User note: You will need to install 'ffmpeg' in the cmd prompt to save the .mpg to disk. Use the following command to install from the conda-forge channel:

```shell
conda install -c conda-forge ffmpeg
```

Uncomment, run the next cell to save the animation to MP4:

In [ ]:
#ani.save("earthdatacloud_animation_GRACEFO.mp4", writer=animation.FFMpegWriter())

### __SMAP Data Analysis__

Normally, we'd pull the entire 2019 data for the SMAP collections as well (recall we only requested 1 month of data earlier in the notebook, due to current system constraints), average the daily SMAP data to monthly means, and compare with the monthly GRACE in a 2019 monthly time series.

For now, let's create the building blocks for that workflow, by creating the 1 monthly mean for SMAP, plotting a spatial map of the SMAP SSS monthly mean to take a quick look at that data, and setting up the 2019 monthly time series plotting (without completing here).

Frist, we can subset the SMAP data as we did above with GRACE. Note the coordinate syntax differs slightly as noted below.

In [ ]:
#SMAP
lat_bnds, lon_bnds = [-2, 6], [180-52, 180-43] #switched lat directions from GRACE, and longitude has positives and negatives
ds_SMAP_subset = ds_SMAP.sel(lat=slice(*lat_bnds), lon=slice(*lon_bnds))
print(ds_SMAP_subset.lat.min().data,
      ds_SMAP_subset.lat.max().data,
      ds_SMAP_subset.lon.min().data,
      ds_SMAP_subset.lon.max().data,)
ds_SMAP_subset

**Important**: remember to use `compute()` so that the `scale_factor` is applied!

We'll also replace fill values with numpy.nan for our next monthly mean computation step.

In [ ]:
# Scale the values to their valid range (compute):
sss_data = ds_SMAP_subset.sss_smap.data.compute()

# Replace fills with numpy nan:
sss_data[sss_data==-9999.] = np.nan

# Replace the data array for the sss monthly mean variable:
ds_SMAP_subset.sss_smap.data = sss_data

print(ds_SMAP_subset.sss_smap)

Now compute monthly means using the convenient xarray `groupby` method and view the minimum and maximum monthly mean values.

In [ ]:
ds_SMAP_subset_sss_momean = ds_SMAP_subset.sss_smap.groupby("time.month").mean(skipna=True)
ds_SMAP_subset_sss_momean.min(skipna=True), ds_SMAP_subset_sss_momean.max(skipna=True)

#### __Plot the April mean SMAP SSS__

Creating a basic plot is easy using the built-in plotting capabilities of xarray:

In [ ]:
#plot SMAP subset
ds_SMAP_subset_sss_momean.sel(month=4).plot()
plt.show()

In [ ]:
sss_min, sss_max = [ds_SMAP_subset.sss_smap.min(skipna=True).compute().data.item(), ds_SMAP_subset.sss_smap.max(skipna=True).compute().data.item()]
sss_min, sss_max

In [ ]:
#A new figure window
fig = plt.figure(figsize=[10,8]) 
ax = fig.add_subplot(1, 1, 1)  # specify (nrows, ncols, axnum)
pmap = plt.axes(projection=ccrs.PlateCarree())

#Necessary Variables for functions
extent = [180-52, 180-43, -2, 6]                                           #lat/lon extents of map
x,y = np.meshgrid(ds_SMAP_subset.lon, ds_SMAP_subset.lat)            #x, y lat/lon values for functions                                
levels = np.linspace(sss_min, sss_max, 10)                                    #number of levels for color differentiation
cmap='viridis'                                                       #color scheme
t=1                                                                 #time to start with
var = ds_SMAP_subset.sss_smap.compute()                                        #variable we will be subsetting from the GRACE-FO data
title = str(pd.to_datetime(ds_SMAP_subset.time[t].values))           #Time of specific time step

#Set up first time step
cont = setup_map(ax, pmap, ds_SMAP_subset, x, y, var, t, cmap, levels, extent) 

#Make a color bar
fig.colorbar(cont, cmap=cmap, boundaries=levels, ticks=levels, 
             orientation='horizontal', label='Sea Surface Salinity (psu)')

#### Animate April mean SMAP SSS

Like we did with GRACE, we can animate the changes for the single month of SMAP data to see how salinty changes over time. 

In [ ]:
#Create animation for April 2019 (change the frame range for different time periods)
ani = animation.FuncAnimation(fig, animate_ts, frames=range(0, ds_SMAP_subset.time.size-1), fargs=(
    ax, pmap, ds_SMAP_subset, x, y, var, t, cmap, levels, extent
),  interval=400)

HTML(ani.to_html5_video())

## __Tutorial Summary and Additional Resources__

The building blocks are now in place to do a longer time series analysis across GRACE and SMAP data, to better understand the relationship between river discharge and sea surface salinity for impact assessment. 

To conclude, we've searched programmatically for data archived in the PO.DAAC Earthdata Cloud over a region and time period of interest, requested the data from the Harmony API, read the data directly into `xarray` from the staged s3 location within AWS `us-west-2` without having to pull the data down into local storage, and performed subsetting and plotting in preparation for a time series analysis. 

There are two more sections of resources below:

1. __Adding river height (pre-SWOT) data to our our analysis__
    - Pre-SWOT data are available through the PO DAAC on-premise location, so you can continue your analysis with data both in and outside of the cloud using OPeNDAP. 

2. __How to set up a Jupyter Notebook running in your own EC2 instance__
    - This same workflow can be achieved outside of the Pangeo BinderHub within an EC2 instance running your personal AWS account. More information and instructions are available below.

___

#### __On-prem hydro data from Pre-SWOT MEaSUREs program__

Data from [**PRESWOT_HYDRO_GRRATS_L2_DAILY_VIRTUAL_STATION_HEIGHTS_V2**](https://podaac.jpl.nasa.gov/dataset/PRESWOT_HYDRO_GRRATS_L2_DAILY_VIRTUAL_STATION_HEIGHTS_V2) are not currently available on the cloud, but we can access via the PO.DAAC's on-prem OPeNDAP service (Hyrax) instead.

<img src="https://podaac.jpl.nasa.gov/Podaac/thumbnails/PRESWOT_HYDRO_GRRATS_L2_DAILY_VIRTUAL_STATION_HEIGHTS_V2.jpg" width="55%">

The guidebook explains the details of the Pre-SWOT MEaSUREs data: https://podaac-tools.jpl.nasa.gov/drive/files/allData/preswot_hydrology/L2/rivers/docs/GRRATS_user_handbookV2.pdf

**Access URL for PO.DAAC on-prem OPeNDAP service**

Identify an appropriate OPeNDAP endpoint through the following steps:

1. Go to the project/mission page on the PO.DAAC portal (e.g. for Pre-SWOT MEaSUREs: https://podaac.jpl.nasa.gov/MEaSUREs-Pre-SWOT)

2. Choose the dataset of interest. Go to the "Data Access" tab of the corresponding dataset landing page, which should like the OPeNDAP access link (for compatible datasets, e.g. for the daily river heights from virtual stations: https://podaac-opendap.jpl.nasa.gov/opendap/allData/preswot_hydrology/L2/rivers/daily/).

3. Navigate to the desired NetCDF file and copy the endpoint (e.g. for our Amazon Basin use case we choose the South America file: https://opendap.jpl.nasa.gov/opendap/allData/preswot_hydrology/L2/rivers/daily/South_America_Amazon1kmdaily.nc).

### Open netCDF file with *xarray*

Open the netCDF dataset via OPeNDAP using *xarray*:

In [ ]:
ds_MEaSUREs = xr.open_dataset('https://opendap.jpl.nasa.gov/opendap/allData/preswot_hydrology/L2/rivers/daily/South_America_Amazon1kmdaily.nc')
print(ds_MEaSUREs)

Our desired variable is height (meters above EGM2008 geoid) for this exercise, which can be subset by distance and time. Distance represents the distance from the river mouth, in this example, the Amazon estuary. Time is between April 8, 1993 and April 20, 2019.

### Plot

**Amazon River heights for March 16, 2018**

Plot the river distances and associated heights on the map at time t=9069:

In [ ]:
fig = plt.figure(figsize=[13,9]) 
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ax.set_extent([-85, -30, -20, 20])
ax.add_feature(cartopy.feature.RIVERS)

plt.scatter(ds_MEaSUREs.lon, ds_MEaSUREs.lat, lw=1, c=ds_MEaSUREs.height[:,9069])
plt.colorbar(label='Interpolated River Heights (m)')
plt.clim(-10,100)

plt.show()

For GRACE-FO, plotting lwe_thickness[107:179,34,69] indicates time, latitude, and longitude indices corresponding to the pixel for the time period 1/2019 to 12/2019 at lat/lon (-0.7, -50). For the 2019 year, measurements of LWE thickness followd expected patterns of high volume of water from the river output into the estuary.

**2011-2019 Seasonality Plots (WIP)**

For GRACE-FO, plotting lwe_thickness[107:179,34,69] indicates time, latitude, and longitude indices corresponding to the pixel for the time period 8/2011 to 12/2019 at lat/lon (-0.7, -50).

In [ ]:
#plot variables associated with river
fig, ax1 = plt.subplots(figsize=[12,7])
#plot river height
ds_MEaSUREs.height[16,6689:9469].plot(color='darkblue')

#plot LWE thickness on secondary axis
ax2 = ax1.twinx()
ax2.plot(subset_GRACE.time[107:179], subset_GRACE.lwe_thickness[107:179,34,69], color = 'darkorange')

ax1.set_xlabel('Date')
ax2.set_ylabel('Land Water Equivalent Thickness (cm)', color='darkorange')
ax1.set_ylabel('River Height (m)', color='darkblue')
ax2.legend(['GRACE-FO'], loc='upper right')
ax1.legend(['Pre-SWOT MEaSUREs'], loc='lower right')

plt.title('Amazon Estuary, 2011-2019 Lat, Lon = (-0.7, -50)')

plt.show()

____

## __Set up for in-region access__


__This notebook must be running within an AWS EC2 instance running in the `us-west-2` region.__

For the live AGU Workshop, our BinderHub instance already takes care of steps 1 and 2, but these instructions are provided so that you can set this up in your own AWS account outside of the workshop.

1. Follow tutorials 01 through 03 of the [NASA Earthdata Cloud Primer](https://earthdata.nasa.gov/learn/user-resources/webinars-and-tutorials/cloud-primer) to set up an EC2 instance within us-west-2. Ensure you are also following step 3 in the ["Jupyter Notebooks on AWS EC2 in 12 (mostly easy) steps"](https://medium.com/@alexjsanchez/python-3-notebooks-on-aws-ec2-in-15-mostly-easy-steps-2ec5e662c6c6) article to set the correct security group settings needed to connect your local port to your EC2’s notebook port thru SSH.

2. Follow the remaining instructions in the Medium article above up until Step 11 (running Jupyter Lab). These instructions include installation of Anaconda3 (including Jupyter Lab) in your ec2 instance. Note the following updates and suggestions:
    * Step 5: Type the following command instead of what is suggested in the article: `ssh -i "tutorialexample.pem" ec2-user@ec2-54-144-47-199.compute-1.amazonaws.com -L 9999:localhost:8888`. This will eliminate the need to create a ssh config file in your home directory (Step 10).
    * As of December 2020, the most current Anaconda3 Linux distribution is: https://repo.anaconda.com/archive/Anaconda3-2020.11-Linux-x86_64.sh
    * The Anaconda installation prompts are not the same as in the article. You will not be prompted to include Anaconda3 in your .bashrc PATH so you can skip to their step 9. Instead select "yes" to initialize Anaconda by running `conda init`. 

Before moving over to Jupyter Lab, set up your Earthdata Login authentication and Harmony access keys:

3. Setup your `~/.netrc` for Earthdata Login in your ec2 instance:

```
cd ~
touch .netrc
echo "machine urs.earthdata.nasa.gov login uid_goes_here password password_goes_here" > .netrc
chmod 0600 .netrc
```

4. Run the following in your ec2 instance terminal window to generate short-term Harmony access keys:

`curl -Ln -bj https://harmony.earthdata.nasa.gov/cloud-access.sh`

5. Set your environment variables based on the keys provided in step 4:

`export AWS_ACCESS_KEY_ID='...
export AWS_SECRET_ACCESS_KEY='...'
export AWS_SESSION_TOKEN='...'
export AWS_DEFAULT_REGION='us-west-2'`

Note that these expire within 8 hours of the script generation.

6. Launch jupyter lab:

`jupyter lab --no-browser`

Copy the URL that begins with `http://localhost:8888` into a browser window. Replace `8888` with `9999`. 

You should now be up and running with JupyterLab in your EC2!
****